# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f780cbd98e0>
├── 'a' --> tensor([[-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309]])
└── 'x' --> <FastTreeValue 0x7f780cbd98b0>
    └── 'c' --> tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                        [-0.5445, -0.8252,  0.2699,  0.8334],
                        [ 0.4808, -2.5085,  1.4038,  1.4455]])

In [4]:
t.a

tensor([[-0.9334, -1.4358,  0.5026],
        [-1.0184, -0.4287,  0.4309]])

In [5]:
%timeit t.a

63.6 ns ± 0.0619 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f780cbd98e0>
├── 'a' --> tensor([[ 0.0110,  0.0477, -1.3354],
│                   [ 0.2677, -0.4541, -1.2953]])
└── 'x' --> <FastTreeValue 0x7f780cbd98b0>
    └── 'c' --> tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                        [-0.5445, -0.8252,  0.2699,  0.8334],
                        [ 0.4808, -2.5085,  1.4038,  1.4455]])

In [7]:
%timeit t.a = new_value

60.8 ns ± 0.0511 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309]]),
    x: Batch(
           c: tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                      [-0.5445, -0.8252,  0.2699,  0.8334],
                      [ 0.4808, -2.5085,  1.4038,  1.4455]]),
       ),
)

In [10]:
b.a

tensor([[-0.9334, -1.4358,  0.5026],
        [-1.0184, -0.4287,  0.4309]])

In [11]:
%timeit b.a

52.2 ns ± 0.0629 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4970, -0.2708, -0.0121],
               [-0.3902,  1.3352,  0.9508]]),
    x: Batch(
           c: tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                      [-0.5445, -0.8252,  0.2699,  0.8334],
                      [ 0.4808, -2.5085,  1.4038,  1.4455]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.0687 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7756e84f10>
├── 'a' --> tensor([[[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]],
│           
│                   [[-0.9334, -1.4358,  0.5026],
│                    [-1.0184, -0.4287,  0.4309]]])
└── 'x' --> <FastTreeValue 0x7f7756e84a60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f780cbd3070>
├── 'a' --> tensor([[-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309],
│                   [-0.9334, -1.4358,  0.5026],
│                   [-1.0184, -0.4287,  0.4309]])
└── 'x' --> <FastTreeValue 0x7f780cbd3250>
    └── 'c' --> tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                        [-0.5445, -0.8252,  0.2699,  0.8334],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69 µs ± 99.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]],
       
               [[-0.9334, -1.4358,  0.5026],
                [-1.0184, -0.4287,  0.4309]]]),
    x: Batch(
           c: tensor([[[-0.2345,  0.8976, -0.2605, -2.4600],
                       [-0.5445, -0.8252,  0.2699,  0.8334],
                       [ 0.4808, -2.5085,  1.4038,  1.4455]],
         

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309],
               [-0.9334, -1.4358,  0.5026],
               [-1.0184, -0.4287,  0.4309]]),
    x: Batch(
           c: tensor([[-0.2345,  0.8976, -0.2605, -2.4600],
                      [-0.5445, -0.8252,  0.2699,  0.8334],
                      [ 0.4808, -2.5085,  1.4038,  1.4455],
                      [-0.2345,  0.8976, -0.2605, -2.4600],
                      [-0.5445, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
